드디어 계산 그래프를 신경망에 적용할 때가 왔다. 여기에서는 신경망을 구성하는 층 각각을 클래스 하나로 구현한다. 우선을 활성화 함수인 ReLU와 Sigmoid 계층을 구현하자.

# 5.5.1 ReLU 계층

활성화 함수로 사용되는 ReLU 함수의 수식은 다음과 같다.

$
y = 
\begin{cases}
x & (x > 0) \\
0 & (x \le 0)
\end{cases}
\qquad$ [식 5.7]

[식 5.7]에서 x에 대한 y의 미분은 [식 5.8]처럼 구한다.

$
{\partial y \over \partial x} = 
\begin{cases}
1 & (x > 0) \\
0 & (x \le 0)
\end{cases}
\qquad$ [식 5.8]

[식 5.8]에서와 같이 순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 흘린다. 반면, 순전파 때의 입력인 x가 0이하면 역전파 때는 하류로 신호를 보내지 않는다.(0을 보낸다.) 계산 그래프로는 다음과 같이 그릴 수 있다.

<img src=images/5_18.png height=100px width=500px>

이제 이 ReLU 계층을 구현해보자. 신경망 계층의 forward()와 backward() 함수는 넘파이 배열을 인수로 받는다고 가정하자.

In [1]:
class ReLULayer:
    def __init__(self):
        pass
    
    def forward(self, x):
        self.x = x
        
        out = (self.x > 0) * 1
            
        return out
    
    def backward(self, dout):
        
        dx = (self.x > 0) * dout
        
        return dx

# 5.5.2 Sigmoid 계층

다음은 시그모이드 함수의 차례이다. 시그모이드 함수는 다음 식을 의미하는 함수이다.

$
y = 
\frac{1}{1+e^{-x}}
\qquad$ [식 5.9]

이를 계산 그래프로 그려보면 다음과 같다.

<img src=images/5_19.png height=100px width=500px>

위 그림에는 "x"와 "+" 노드 이외에 "exp"와 "/"노드가 새롭게 등장했다. "exp"노드는 $y = e^{x}$를 수행하고, "/"노드는 $y = {1 \over x}$를 수행한다.<br>
그림과 같이 [식 5.9]의 계산은 국소적 계산의 전파로 이루어진다. 이제 그림의 역전파를 알아보자. 여기에서는 역전파의 흐름을 오른쪽에서 왼쪽으로 한 단계씩 짚어보자.

#### 1단계

"/"노드, 즉 $y = {1 \over x}$을 미분하면 다음의 식이 된다.

$
\begin{matrix}
{\partial y \over \partial x}
&=& -{1 \over x^2} \\
&=& -y^2
\end{matrix}
\qquad$ [식 5.10]

[식 5.10]에 따르면 역전파 때는 상류에서 흘러온 값에 $-y^2$을 곱해서 하류로 전달한다. 계산 그래프에서는 다음과 같다.

<img src=images/5_20_1.png height=100px width=500px>

#### 2단계
"+"노드는 상류의 값을 여과 없이 하류로 내보내는 것이 전부이다. 계산 그래프로는 다음과 같다.

<img src=images/5_20_2.png height=100px width=500px>

#### 3단계
"exp"노드는  $y = e^x$를 수행하며, 그 미분은 다음과 같다.

$
{\partial y \over \partial x} = 
e^x
\qquad$ [식 5.11]

계산 그래프에서는 상류의 값에 순전파 때의 출력을 곱해 하류로 전파한다.

<img src=images/5_20_3.png height=100px width=500px>

#### 4단계
"x"노드는 순전파 때의 값을 서로 바꿔 곱한다. 이 예에서는 -1을 곱한다.

<img src=images/5_20_0.png height=100px width=500px>

이상으로 Sigmoid 계층의 역전파를 계산 그래프로 완성하였다. 역전파의 최종출력인 ${\partial L \over \partial y}y^2e^{-x}$가 하류 노드로 전파된다. 여기에서 ${\partial L \over \partial y}y^2e^{-x}$를 순전파의 입력인 x와 출력 y만으로 계산할 수 있다. 그래서 위 계산 그래프의 중간 과정을 모두 묶어 다음과 같이 단순한 "sigmoid" 노드 하나로 대체할 수 있다.

<img src=images/5_21.png height=100px width=500px>

위의 서로 다른 두 그림의 결과는 똑같다. 그러나 간소화 버전은 역전파 과정의 중간 계산들을 생략할 수 있어 더 효율적인 계산이라 말할 수 있다. 또, 노드를 그룹화하여 Sigmoid 계층의 세세한 내용을 노출하지 않고 입력과 출력에만 집중할 수 있다는 것도 중요한 포인트이다.
또한, ${\partial L \over \partial y}y^2e^{-x}$는 다음처럼 정리해서 쓸 수 있다.

$
\begin{matrix}
{\partial L \over \partial y}y^2e^{-x}
&=& {\partial L \over \partial y}{1 \over (1+e^{-x})^2}e^{-x} \\
&=& {\partial L \over \partial y}{1 \over 1+e^{-x}}{e^{-x} \over 1+e^{-x}} \\
&=& {\partial L \over \partial y}y(1-y)
\end{matrix}
\qquad$ [식 5.12]

이처럼 Sigmoid 계층의 역전파는 순전파의 출력 y만으로 계산할 수 있다.

<img src=images/5_22.png height=100px width=500px>

그럼 이 Sigmoid 계층을 파이썬으로 구현해보자

In [4]:
class SigmoidLayer:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        
        out = 1 / (1 + np.exp(-x))
        
        self.out = out
        
        return out
    
    def backward(self, dout):
        
        dx = dout * self.out * (1.0 - self.out)
        
        return dx